In [2]:
import pandas as pd
import requests
import psycopg2
import os

In [3]:
def update_expected_points_no_availability():
    """
    Updates the 'expectedPoints' column in the database using predictions from a CSV file
    """
    # Fetch the latest gameweek number
    URL = "https://fantasy.premierleague.com/api/event-status/"
    lastUpdate = requests.get(URL).json()["status"][0]["event"]

    df = pd.read_csv("predictions/all_positions_points.csv")
    rows = ["ID"]

    # Get points for the next 5 gameweeks
    for i in range(lastUpdate + 1, lastUpdate + 6):
        rows.append(f"{i}_Pts")

    points = df[rows]

    conn = psycopg2.connect(
        database=os.environ["DB_NAME"],
        user=os.environ["DB_USER"],
        host=os.environ["DB_HOST"],
        password=os.environ["DB_PASSWORD"],
        port=os.environ["PORT"]
    )
    
    cur = conn.cursor()

    for row in points.iterrows():
        id = int(row[1].iloc[0])

        arr = "{" + ", ".join(row[1][1:].apply(str)) + "}"
        query = f"UPDATE players SET \"expectedPoints\"=\"expectedPoints\"[0:{lastUpdate}]||'{arr}' WHERE id={id}"

        print(query)
        cur.execute(query)

    conn.commit()
    cur.close()
    conn.close()

update_expected_points_no_availability()

KeyError: 'DB_NAME'